<img src="https://raw.githubusercontent.com/maverick-faus/Files/master/DeepDeamon2.png" alt="drawing" width="100px"/>
# Red 4B - v1.2
TT: Deep Deep Dishes  
Food recommendation System  
ESCOM  
By Faus - 2K18


In [1]:
!pip install -U tensorboardcolab
from tensorboardcolab import *

  Running setup.py bdist_wheel for tensorboardcolab ... - done
  Stored in directory: /root/.cache/pip/wheels/ab/74/02/cda602d1dc28b2f12eab313c49b9bfa14d6371326bc2590e06
Successfully built tensorboardcolab


Using TensorFlow backend.


In [2]:
!pip install PyDrive
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from tensorboardcolab import *


    100% |████████████████████████████████| 993kB 8.9MB/s 
  Running setup.py bdist_wheel for PyDrive ... - \ done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleus

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [5]:
!ls drive

'Advanced Neural Networks'
'Artículos Varios TT'
'BBVA Hack - Chicos que lloran'
'Books Wishlist 2K18.ods'
'Books Wishlist 2K18.xlsx'
'Books Wishlist 2K18.xlsx.ods'
 Chatboot
 Classroom
 CodigoTT
'COMUNICADO Patish Complemento de Pago a sus proveedores.docx'
'COMUNICADO Patish Complemento de Pago a sus proveedores.odt'
'Copia de Red4B.ipynb'
'Curso DNN'
'Deep Daemon'
 deepdeep.backup
'Discurso excelencia.odt'
'Documento sin título.odt'
'Documentos Personales '
'Documento TT1'
'Documento TT1 (3a7c594f)'
 DriveC
'Hoja de cálculo sin título.ods'
 Imagenet_Hamburguer.ods
 IMG_2364Trim.mp4
'Prueba de Escritorio - DDD.ods'
'Publicidad dirigida mediante redes neuronales profundas.pdf'
'Retorno Inversiones Jose Shedid Merhy.xlsx'
'Retorno Inversiones Jose Shedid Merhy.xlsx.ods'
'Script Presentación TT1.odt'
'TT1 - Previos'
 TT2
 TT2_W2VReporteFinalFinal.docx
 Untitled0.ipynb


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
fileIdLabelmap = '1lCUmg2TZsB_xduCL3ykBWWk8EitIDnTg'
#label map
downloaded1 = drive.CreateFile({'id': fileIdLabelmap})
downloaded1.GetContentFile("4B_train.pickle")

In [8]:
!ls

4B_train.pickle  adc.json  drive  sample_data


In [0]:
import tensorflow as tf
import numpy as np
import pickle
from sklearn.metrics import confusion_matrix
#from tensorboardcolab import *

In [10]:
print (tf.VERSION)

1.12.0-rc2


In [0]:
def conv2d(x, W,name,padd,strid=[1,1,1,1]):
    #El stride de esa función no reduce el tamaño de la imagen
    return tf.nn.conv2d(x, W, strides=strid, padding=padd,name=name)

def maxpool2d(x,ks,st):
    #           El st de esta función reduce la imagen a la mitad
    return tf.nn.max_pool(x, ksize=ks, strides=st, padding='SAME')

In [0]:
def reset_graph():
    #Limpiamos la gráfic
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

In [0]:
def deep_neural_convolutional_class(
    batch_size=20,
    image_size=[200,200],
    Drop_prob=1.0,
    learning_rate = 1e-3,
    n_nodes_hl0 = 2000,
    n_nodes_hl1 = 1000,
    n_nodes_hl2 = 500,
    n_nodes_hl3 = 100,
    n_classes=17
    ):
    
    reset_graph()
    #Place holder de entrada 
    x= tf.placeholder(tf.float32,[batch_size,image_size[0],image_size[1],3], name='placeholder_img_entrada')
    y=tf.placeholder(tf.float32,name='placeholder_one_hot')
  
    #Diccionario de pesos convolucionales 
    with tf.name_scope('pesos_bias') as scope1:
        weigths={"w_conv1":tf.Variable(tf.random_normal([5,5,3,32]),name='Pesos_1_32'),
                 "w_conv2":tf.Variable(tf.random_normal([5,5,32,64]),name='Pesos_1_64'),
                 "w_conv3":tf.Variable(tf.random_normal([3,3,64,128]),name='Pesos_1_128'),   
                 "w_conv4":tf.Variable(tf.random_normal([5,5,128,256]),name='Pesos_1_256'),
                 "w_conv5":tf.Variable(tf.random_normal([5,5,256,512]),name='Pesos_1_512'),
                 "w_conv6":tf.Variable(tf.random_normal([5,5,512,1024]),name='Pesos_1_1024'),
                }
        #Diccionario de bias
        biases={"b_conv1":tf.Variable(tf.random_normal([32]),name='Bias_1_32'),
                "b_conv2":tf.Variable(tf.random_normal([64]),name='Bias_1_64'),
                "b_conv3":tf.Variable(tf.random_normal([128]),name='Bias_1_128'),
                "b_conv4":tf.Variable(tf.random_normal([256]),name='Bias_1_256'),
                "b_conv5":tf.Variable(tf.random_normal([512]),name='Bias_1_512'),
                "b_conv6":tf.Variable(tf.random_normal([1024]),name='Bias_1_1024'),
               }

    #Extractor de características
    with tf.name_scope('capas_conv') as scope2:
        conv1=tf.nn.relu(conv2d(x,weigths["w_conv1"],'Capa_Conv_1','SAME')+biases["b_conv1"],name='Func_relu_1')
        conv1=tf.nn.dropout(conv1,Drop_prob)
        conv1=maxpool2d(conv1,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 100x100x32
        print(conv1)

        conv2=tf.nn.relu(conv2d(conv1,weigths["w_conv2"],'Capa_Conv_2','SAME')+biases["b_conv2"],name='Func_relu_2')
        conv2=tf.nn.dropout(conv2,Drop_prob)
        conv2=maxpool2d(conv2,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 50x50x64
        print(conv2)

        conv3=tf.nn.relu(conv2d(conv2,weigths["w_conv3"],'Capa_Conv_3','VALID')+biases["b_conv3"],name='Func_relu_3')
        conv3=tf.nn.dropout(conv3,Drop_prob)
        conv3=maxpool2d(conv3,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 24x24x128
        print(conv3)

        conv4=tf.nn.relu(conv2d(conv3,weigths["w_conv4"],'Capa_Conv_4','SAME')+biases["b_conv4"],name='Func_relu_4')
        conv4=tf.nn.dropout(conv4,Drop_prob)
        conv4=maxpool2d(conv4,ks=[1,2,2,1],st=[1,2,2,1])
        #imagen resultante de 12x12x256
        print(conv4)

        conv5=tf.nn.relu(conv2d(conv4, weigths["w_conv5"],'Capa_Conv_5','SAME')+biases["b_conv5"],name='Func_relu_5')
        conv5=tf.nn.dropout(conv5,Drop_prob)
        conv5=maxpool2d(conv5,ks=[1,2,2,1],st=[1,2,2,1])
        #vector para clasificar de 6x6x512
        print(conv5)

        #Embeding, son las caracteristicas fonales que se pasarán al MLP o red completamente conectada para clasifiacar
        embdeding=tf.reshape(conv5,[batch_size,6*6*512],name='Embeding')
        print(embdeding)
    
    #Red perceptron, declaración de capas, son diccionarios de pesos y bias.
    with tf.name_scope('capas_clasificador') as scope3:
        hidden_0_layer = {'weights':tf.Variable(tf.random_normal([6*6*512, n_nodes_hl0]),name='Capa_oculta_pesos_0'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl0]),name='Capa_oculta_bias_0')}

        hidden_1_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl0, n_nodes_hl1]),'Capa_oculta_pesos_1'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl1]),name='Capa_oculta_bias_1')}

        hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2]),'Capa_oculta_pesos_2'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl2]),name='Capa_oculta_bias_2')}
        
        hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3]),'Capa_oculta_pesos_3'),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl3]),name='Capa_oculta_bias_3')}

        output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes]),'Capa_salida_pesos'),
                        'biases':tf.Variable(tf.random_normal([n_classes]),name='Capa_salida_bias'),}
    
    #W*P + B 
    with tf.name_scope('op_clasificador') as scope4:
        
        l0 = tf.add(tf.matmul(embdeding,hidden_0_layer['weights'],name='Matmul_l0'), hidden_0_layer['biases'],name='Suma_Pesos_Bias_0')
        l0 = tf.nn.relu(l0,name='l0_relu_0')

        l1 = tf.add(tf.matmul(l0,hidden_1_layer['weights'],name='Matmul_l1'), hidden_1_layer['biases'],name='Suma_Pesos_Bias_1')
        l1 = tf.nn.relu(l1,name='l1_relu_1')

        l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights'],name='Matmul_l2'), hidden_2_layer['biases'],name='Suma_Pesos_Bias_2')
        l2 = tf.nn.relu(l2,name='l2_relu_2')
        
        l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights'],name='Matmul_l3'), hidden_3_layer['biases'],name='Suma_Pesos_Bias_3')
        l3 = tf.nn.relu(l3,name='l3_relu_3')

        output = tf.matmul(l3,output_layer['weights'],name='Matmul_out') + output_layer['biases']
    
    
    
    
    
    # Declarando la funcion de costo y entrenamiento
    #Reduce mean, reduce la dimencion del tensor en un promedio es decir hace el promedio del costo o error
    y1,y2,y3,y4= tf.split(y,[5,5,4,3],1) #PaRTIR EL VECTOR EN CADA ONE HOT 
    print(y1)
    print(y2)
    print(y3)
    print(y4)
    print("----")
    output1,output2,output3,output4= tf.split(output, [5,5,4,3], 1)
    print(output)
    print(output1)
    print(output2)
    print(output3)
    print(output4)
    
    
    with tf.name_scope('costo_y_optimizador') as scope5:
        cost1 = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=output1,labels=y1,name='softmax_cross_entropy_with_logits1'),name='reduce_mean1')
        cost2 = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=output2,labels=y2,name='softmax_cross_entropy_with_logits2'),name='reduce_mean2')
        cost3 = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=output3,labels=y3,name='softmax_cross_entropy_with_logits3'),name='reduce_mean3')
        cost4 = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=output4,labels=y4,name='softmax_cross_entropy_with_logits4'),name='reduce_mean4')
        cost = cost1+cost2+cost3+cost4
        optimizer = tf.train.AdamOptimizer(name='Adam',learning_rate=learning_rate).minimize(cost,name='minimo')
    
    
    tf.summary.scalar("costo",cost)
    
    
    #correct = tf.equal(tf.argmax(output,1),tf.argmax(y,1),name='valores_correctos')
    #accuracy = tf.reduce_mean(tf.cast(correct,'float'),name='Promedio_exactitud') #porcentaje de error

    
    correct_1 = tf.equal(tf.argmax(output1,1),tf.argmax(y1,1),name='valores_correctos1')
    accuracy_1 = tf.reduce_mean(tf.cast(correct_1,'float'),name='Promedio_exactitud1') #porcentaje de error
    
    correct_2 = tf.equal(tf.argmax(output2,1),tf.argmax(y2,1),name='valores_correctos2')
    accuracy_2 = tf.reduce_mean(tf.cast(correct_2,'float'),name='Promedio_exactitud2') #porcentaje de error
    
    correct_3 = tf.equal(tf.argmax(output3,1),tf.argmax(y3,1),name='valores_correctos3')
    accuracy_3 = tf.reduce_mean(tf.cast(correct_3,'float'),name='Promedio_exactitud3') #porcentaje de error
    
    correct_4 = tf.equal(tf.argmax(output4,1),tf.argmax(y4,1),name='valores_correctos4')
    accuracy_4 = tf.reduce_mean(tf.cast(correct_4,'float'),name='Promedio_exactitud4') #porcentaje de error

    
    avg=accuracy_1*0.3+accuracy_2*0.3+accuracy_3*0.2+accuracy_4*0.2
    
    
    
    
    tf.summary.scalar("average",avg)
    summaries = tf.summary.merge_all()
    
    return dict(
        x = x,
        y=y,
        embeding=conv5,
        output=output,
        saver = tf.train.Saver(),
        total_loss = cost,
        train_step = optimizer,
        summaries = summaries,
        average = avg,
        accuracy_precio = accuracy_1,
        accuracy_estilo = accuracy_2,
        accuracy_pan = accuracy_3,
        accuracy_side = accuracy_4,
        loss_precio = cost1,
        loss_estilo = cost2,
        loss_pan = cost3,
        loss_side = cost4
    )

In [14]:
deep_neural_convolutional_class()

Tensor("capas_conv/MaxPool:0", shape=(20, 100, 100, 32), dtype=float32)
Tensor("capas_conv/MaxPool_1:0", shape=(20, 50, 50, 64), dtype=float32)
Tensor("capas_conv/MaxPool_2:0", shape=(20, 24, 24, 128), dtype=float32)
Tensor("capas_conv/MaxPool_3:0", shape=(20, 12, 12, 256), dtype=float32)
Tensor("capas_conv/MaxPool_4:0", shape=(20, 6, 6, 512), dtype=float32)
Tensor("capas_conv/Embeding:0", shape=(20, 18432), dtype=float32)
Tensor("split:0", dtype=float32)
Tensor("split:1", dtype=float32)
Tensor("split:2", dtype=float32)
Tensor("split:3", dtype=float32)
----
Tensor("op_clasificador/add:0", shape=(20, 17), dtype=float32)
Tensor("split_1:0", shape=(20, 5), dtype=float32)
Tensor("split_1:1", shape=(20, 5), dtype=float32)
Tensor("split_1:2", shape=(20, 4), dtype=float32)
Tensor("split_1:3", shape=(20, 3), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_

{'accuracy_estilo': <tf.Tensor 'Promedio_exactitud2:0' shape=() dtype=float32>,
 'accuracy_pan': <tf.Tensor 'Promedio_exactitud3:0' shape=() dtype=float32>,
 'accuracy_precio': <tf.Tensor 'Promedio_exactitud1:0' shape=() dtype=float32>,
 'accuracy_side': <tf.Tensor 'Promedio_exactitud4:0' shape=() dtype=float32>,
 'average': <tf.Tensor 'add_2:0' shape=() dtype=float32>,
 'embeding': <tf.Tensor 'capas_conv/MaxPool_4:0' shape=(20, 6, 6, 512) dtype=float32>,
 'loss_estilo': <tf.Tensor 'costo_y_optimizador/reduce_mean2:0' shape=() dtype=float32>,
 'loss_pan': <tf.Tensor 'costo_y_optimizador/reduce_mean3:0' shape=() dtype=float32>,
 'loss_precio': <tf.Tensor 'costo_y_optimizador/reduce_mean1:0' shape=() dtype=float32>,
 'loss_side': <tf.Tensor 'costo_y_optimizador/reduce_mean4:0' shape=() dtype=float32>,
 'output': <tf.Tensor 'op_clasificador/add:0' shape=(20, 17) dtype=float32>,
 'saver': <tensorflow.python.training.saver.Saver at 0x7f2b915bc0f0>,
 'summaries': <tf.Tensor 'Merge/MergeSumma

In [0]:
def train_img_encoder(G,nameBatch,numbatch, num_epochs,learning_rate = 1e-2, 
                      num_steps = 15, batch_size = 20, verbose = True, save=False,checkpoint=False):
    #tf.set_random_seed(2345)
    print('Start training')
    total_loss = 0
    with tf.Session() as sess:
        
        writer = tf.summary.FileWriter("./CNNClass")
        tf.summary.FileWriter.add_graph(writer,sess.graph)
        
        sess.run(tf.global_variables_initializer()) ###########
        print('Enter here3')
        if checkpoint:
            ENCname="./"+checkpoint+".ckpt"
            G['saver'].restore(sess, ENCname)
        training_losses = []
        for epoch in range(num_epochs):
            epoch_loss = 0

            for key in range(numbatch):
                data=np.array(pickle.load(open(nameBatch+".pickle","rb")))
                dim,var=data.shape
                for j in range(int(dim/batch_size)): #Unir cada etiqueta que viene por separado a un vector de 17 posiciones
                    epoch_x=data[batch_size*(j):batch_size*(j+1),0].tolist()
                    epoch_Y1=data[batch_size*(j):batch_size*(j+1),1].tolist()
                    epoch_Y2=data[batch_size*(j):batch_size*(j+1),2].tolist()
                    epoch_Y3=data[batch_size*(j):batch_size*(j+1),3].tolist()
                    epoch_Y4=data[batch_size*(j):batch_size*(j+1),4].tolist()
                    new_Y =[]
                    for k in range(len(epoch_Y1)):
                      new_Y.append(epoch_Y1[k]+epoch_Y2[k]+epoch_Y3[k]+epoch_Y4[k])

                    feed_dict={G['x']: epoch_x,
                               G['y']: new_Y,
                              }
                
                    total_loss,_,summ,avg,a_precio,a_estilo,a_pan,a_side,l_precio,l_estilo,l_pan,l_side = sess.run([G["total_loss"],
                                                                                                                         G["train_step"],
                                                                                                                         G["summaries"],
                                                                                                                         G["average"],
                                                                                                                         G["accuracy_precio"],
                                                                                                                         G["accuracy_estilo"],
                                                                                                                         G["accuracy_pan"],
                                                                                                                         G["accuracy_side"],
                                                                                                                         G["loss_precio"],
                                                                                                                         G["loss_estilo"],
                                                                                                                         G["loss_pan"],
                                                                                                                         G["loss_side"]],feed_dict)

                epoch_loss += total_loss
                writer.add_summary(summ,epoch)
                
                tbc.save_value("Evaluación de recomendación", "average", epoch, avg)
                tbc.save_value("Accuracy/Precio", "accuracyP", epoch, a_precio)
                tbc.save_value("Accuracy/Estilo", "accuracyE", epoch, a_estilo)
                tbc.save_value("Accuracy/Pan", "accuracyB", epoch, a_pan)
                tbc.save_value("Accuracy/Guarnición", "accuracyS", epoch, a_side)
                
                tbc.save_value("Loss", "loss", epoch, epoch_loss)
                tbc.save_value("Loss/Precio", "lossP", epoch, l_precio)
                tbc.save_value("Loss/Estilo", "lossE", epoch, l_estilo)
                tbc.save_value("Loss/Pan", "lossB", epoch, l_pan)
                tbc.save_value("Loss/Guarnición", "lossS", epoch, l_side)
                
                tbc.flush_line("loss")
                tbc.flush_line("lossP")
                tbc.flush_line("lossE")
                tbc.flush_line("lossB")
                tbc.flush_line("lossS")
                
                tbc.flush_line("average")
                tbc.flush_line("accuracyP")
                tbc.flush_line("accuracyE")
                tbc.flush_line("accuracyB")
                tbc.flush_line("accuracyS")               
                
            if verbose:
                print("+------------------+\n|                  |\n|      Epoch:      |\n|      ",epoch,"         |\n|                  |\n|      Loss:       |\n|  %e    |\n|                  |\n+------------------+\n (\__/) | |\n (●ㅅ●) | |  \n /    っ    \n"%(epoch_loss))
                #print("Epoch", epoch, "  Loss:", epoch_loss, "  Accurracy:", accuracy)
            training_losses.append(epoch_loss)

        if isinstance(save, str):
            ENCname="drive/CodigoTT/4B/ckpt4B/"+save+".ckpt"
            G['saver'].save(sess, ENCname)
    return training_losses

In [33]:
tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://98b4c13e.ngrok.io


In [34]:
path = '4B_train' 
encoder=deep_neural_convolutional_class(batch_size = 20)
train_img_encoder(encoder,path,1,num_epochs=40,save="4B3",batch_size = 20,verbose = True)

Tensor("capas_conv/MaxPool:0", shape=(20, 100, 100, 32), dtype=float32)
Tensor("capas_conv/MaxPool_1:0", shape=(20, 50, 50, 64), dtype=float32)
Tensor("capas_conv/MaxPool_2:0", shape=(20, 24, 24, 128), dtype=float32)
Tensor("capas_conv/MaxPool_3:0", shape=(20, 12, 12, 256), dtype=float32)
Tensor("capas_conv/MaxPool_4:0", shape=(20, 6, 6, 512), dtype=float32)
Tensor("capas_conv/Embeding:0", shape=(20, 18432), dtype=float32)
Tensor("split:0", dtype=float32)
Tensor("split:1", dtype=float32)
Tensor("split:2", dtype=float32)
Tensor("split:3", dtype=float32)
----
Tensor("op_clasificador/add:0", shape=(20, 17), dtype=float32)
Tensor("split_1:0", shape=(20, 5), dtype=float32)
Tensor("split_1:1", shape=(20, 5), dtype=float32)
Tensor("split_1:2", shape=(20, 4), dtype=float32)
Tensor("split_1:3", shape=(20, 3), dtype=float32)
Start training
Enter here3
+------------------+
|                  |
|      Epoch:      |
|       0          |
|                  |
|      Loss:       |
|  1.026257e+15    |

[1026256934010880.0,
 469408887603200.0,
 364955215855616.0,
 243584070582272.0,
 173544361164800.0,
 163781430738944.0,
 122657655750656.0,
 95478624550912.0,
 110059778473984.0,
 86730522755072.0,
 43825036787712.0,
 30684496265216.0,
 24110736343040.0,
 37255603290112.0,
 43019176771584.0,
 41186068791296.0,
 20872706392064.0,
 18711035510784.0,
 7348003274752.0,
 25017060425728.0,
 28358811320320.0,
 11799491510272.0,
 14621319626752.0,
 7903244189696.0,
 10647260626944.0,
 4316366897152.0,
 6719044321280.0,
 8183345053696.0,
 14138760757248.0,
 6744671518720.0,
 2015743967232.0,
 2669812908032.0,
 6976965705728.0,
 1386376331264.0,
 5753074089984.0,
 14589863395328.0,
 11543428202496.0,
 5869965672448.0,
 7773431529472.0,
 12062337007616.0]

In [0]:
!ls